In [59]:
import torch
import torch.utils.data
import torchvision
from glob import glob
import json
import os
from pathlib import Path
import random
from PIL import Image

In [11]:
config = json.load(open(os.path.expanduser("~/.thesis.conf")))
db_folder = Path(config['datasets']) / Path("archive_org/") 
identifier = "pointlinetoplane00kand"
os.chdir(str(db_folder))

In [120]:
class ArchiveOrgDataset(torch.utils.data.Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.jp2 = glob(str(path / '*'/ '*' / '*.jp2'))
        self.transform = transform
        
    def __len__(self):
        return len(self.jp2)

    def __getitem__(self, index):
        assert index < len(self.jp2)
        img = Image.open(self.jp2[index])
        if self.transform:
            img = self.transform(img)
        return img

class RotateTask(object):
    def __init__(self, interpolation=Image.BILINEAR,  resample=False, expand=False, center=None):
        self.interpolation = interpolation
        self.resample = resample
        self.expand = expand
        self.center = None
        self.directions = {0: 0, 1: 90, 2: 180, 3: 270 }
    def __call__(self, img):
        d = random.randint(0,3)
        rimg = img.rotate(self.directions[d], resample=Image.BILINEAR)
        rimg = torchvision.transforms.functional.to_tensor(rimg)
        return dict(x=rimg, label=d)
        

In [121]:
createRotateTask = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(64,scale=(0.8,1), ratio=(1,1)),
    RotateTask()
])
aset = ArchiveOrgDataset(db_folder, transform=createRotateTask)

In [123]:
aset[2]

{'label': 0, 'x': 
 (0 ,.,.) = 
   0.8196  0.8275  0.8314  ...   0.7843  0.7804  0.7529
   0.8196  0.8275  0.8314  ...   0.7843  0.7804  0.7569
   0.8235  0.8314  0.8314  ...   0.7843  0.7804  0.7569
            ...             ⋱             ...          
   0.8118  0.8235  0.8196  ...   0.7922  0.7843  0.7765
   0.8078  0.8157  0.8196  ...   0.7843  0.7843  0.7686
   0.8039  0.8118  0.8118  ...   0.7804  0.7804  0.7686
 
 (1 ,.,.) = 
   0.7412  0.7529  0.7529  ...   0.7020  0.6941  0.6588
   0.7412  0.7529  0.7529  ...   0.7059  0.6980  0.6627
   0.7490  0.7529  0.7529  ...   0.7059  0.6941  0.6627
            ...             ⋱             ...          
   0.7176  0.7294  0.7373  ...   0.7098  0.6980  0.6824
   0.7098  0.7216  0.7294  ...   0.7059  0.6902  0.6745
   0.7059  0.7176  0.7294  ...   0.6980  0.6863  0.6706
 
 (2 ,.,.) = 
   0.6039  0.6118  0.6157  ...   0.5686  0.5569  0.5137
   0.6039  0.6157  0.6157  ...   0.5686  0.5569  0.5137
   0.6118  0.6157  0.6196  ...   0.5686  0

In [108]:
data_loader = torch.utils.data.DataLoader(aset,batch_size=4)

Init signature: torchvision.transforms.ToTensor(self, /, *args, **kwargs)
Source:
class ToTensor(object):
    """Convert a ``PIL Image`` or ``numpy.ndarray`` to tensor.

    Converts a PIL Image or numpy.ndarray (H x W x C) in the range
    [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0].
    """

    def __call__(self, pic):
        """
        Args:
            pic (PIL Image or numpy.ndarray): Image to be converted to tensor.

        Returns:
            Tensor: Converted image.
        """
        return F.to_tensor(pic)

File:           ~/anaconda/envs/ml-software/lib/python3.5/site-packages/torchvision/transforms/transforms.py
Type:           type